In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning) # ignores warnings from pd.append()

In [45]:
# column names
col_names = ['Titel', 'Bevilliget beløb', 'Modtager', 'Institution', 'Virkemidler', 'Område', 'År', 'Beskrivelse']

# create an Empty DataFrame object
df = pd.DataFrame(columns = col_names)

In [46]:
URL = "https://dff.dk/forskningsprojekter?b_start:int="

for i in range(0, 10, 10):

    print("Scraping page: "+ str(int(float(i/10)))) # prints page being scraped at the moment

    page = requests.get(URL+str(i))
    soup = BeautifulSoup(page.text, "html.parser")

    # Load page as html
    bevillinger = soup.find_all("div", class_ = "result-item")

    # Retrieve all grants from first page
    for bevilling in bevillinger:
        titel = bevilling.find("h2", class_ = "result-title").text.strip()
        beløb = bevilling.find("div", class_ = "col-sm-2 text-right result-amount").text.replace("Bevilget beløb", "").strip()
        
        modtager_liste = bevilling.find("div", class_ = "col-xs-6 col-sm-12").text.strip().split(sep="\n\n") # remove whitespaces and split based on "\n\n"
        modtager_liste = [s.strip() for s in modtager_liste] # strip whitespace for both Modtager and Institution

        try:
            modtager = modtager_liste[0]
            institution = modtager_liste[1]
        except IndexError:
            modtager = modtager_liste[0]
            institution = ""
        
        info_liste = bevilling.find("ul", class_ = "listing-horizontal").text.strip().split(sep="\n\n") # remove whitespaces and split based on "\n\n"
        info_liste = [s.strip() for s in info_liste] # strip whitespace for both Virkemiddel, Område and År

        virkemidler = info_liste[0]
        område = info_liste[1]
        periode = info_liste[2]

        beskrivelse = bevilling.find("p", class_ = "col-md-6")

        # append rows to an empty DataFrame
        df = df.append({'Titel'             : titel,
                        'Bevilliget beløb'  : beløb,
                        'Modtager'          : modtager,
                        'Institution'       : institution,
                        'Virkemidler'       : virkemidler,
                        'Område'            : område,
                        'År'                : periode,
                        'Beskrivelse'       : beskrivelse},
                        ignore_index = True)

Scraping page: 0


In [47]:
# print dataframe
df

,Titel,Bevilliget beløb,Modtager,Institution,Virkemidler,Område,År,Beskrivelse
0,Absence of information in decision-making proc...,2.533.013 kr,Sille Obelitz Søe,Københavns Universitet,Inge Lehmann,Tematisk,2022,[I menneskelig interaktion kommunikeres inform...
1,Are transient awakenings during sleep a key dr...,2.879.617 kr,Celia Kjærby,Københavns Universitet,Inge Lehmann,Tematisk,2022,[For at bevare et langt liv og en sund alderdo...
2,Changing Professionals’ Attitudes to Volunteer...,2.766.880 kr,Mette Kjærgaard Thomsen,Syddansk Universitet,Inge Lehmann,Tematisk,2022,"[Ny dansk forskning viser, at mange fagprofess..."
3,Comorbidity networks of mental disorders and g...,2.776.932 kr,Karen-Inge Karstoft,Københavns Universitet,Inge Lehmann,Tematisk,2022,[Mennesker med en psykisk lidelse er i øget ri...
4,Cultivating Women Tech Founders: A Communities...,2.834.347 kr,Shuang Lu,Aarhus Universitet,Inge Lehmann,Tematisk,2022,[Flere kvindelige iværksættere i tech-branchen...
5,DarkDNA - unidentified eDNA sequences as ecolo...,2.879.406 kr,Eva Egelyng Sigsgaard,Aarhus Universitet,Inge Lehmann,Tematisk,2022,[I den nuværende biodiversitetskrise er det af...
6,Digital Alternative News Use in Denmark (Alter...,2.879.859 kr,Eva-Maria Laura Anna Mayerhöffer,Roskilde Universitet,Inge Lehmann,Tematisk,2022,[Projektet vil gennemføre den første omfattend...
7,Fibre in Foods: Wheat varieties for enhanced h...,2.874.486 kr,Ourania Gouseti,Københavns Universitet,Inge Lehmann,Tematisk,2022,[Efterhånden som diætrelaterede sygdomme som f...
8,Finding the interaction point of single gamma ...,2.879.888 kr,Rosana Martinez Turtos,Aarhus Universitet,Inge Lehmann,Tematisk,2022,"[Scintillatorer er materialer, som udsender et..."
9,Functional ecosystem changes in the marine Arc...,2.879.782 kr,Heike Hildegard Zimmermann,GEUS,Inge Lehmann,Tematisk,2022,[En af de største udfordring for den naturvide...


In [52]:
# convert column År to numeric
df["År"] = pd.to_numeric(df["År"])

# convert column Bevilliget beløb to numeric
df["Bevilliget beløb"] = df["Bevilliget beløb"].str.replace("kr", "").str.strip()
df["Bevilliget beløb"] = df["Bevilliget beløb"].str.replace(".", "")
df["Bevilliget beløb"] = pd.to_numeric(df["Bevilliget beløb"])

# 
df['Beskrivelse'] = df['Beskrivelse'].replace('<p class="col-md-6"', '')
df['Beskrivelse'] = df['Beskrivelse'].replace('</p>', '')

AttributeError: Can only use .str accessor with string values!

In [53]:
# save to csv
df.to_csv('dff.csv', sep=',', header=True, index=False)

# import csv
df_new = pd.read_csv('dff.csv')
df_new

,Titel,Bevilliget beløb,Modtager,Institution,Virkemidler,Område,År,Beskrivelse
0,Absence of information in decision-making proc...,2533013,Sille Obelitz Søe,Københavns Universitet,Inge Lehmann,Tematisk,2022,"<p class=""col-md-6"">I menneskelig interaktion ..."
1,Are transient awakenings during sleep a key dr...,2879617,Celia Kjærby,Københavns Universitet,Inge Lehmann,Tematisk,2022,"<p class=""col-md-6"">For at bevare et langt liv..."
2,Changing Professionals’ Attitudes to Volunteer...,2766880,Mette Kjærgaard Thomsen,Syddansk Universitet,Inge Lehmann,Tematisk,2022,"<p class=""col-md-6"">Ny dansk forskning viser, ..."
3,Comorbidity networks of mental disorders and g...,2776932,Karen-Inge Karstoft,Københavns Universitet,Inge Lehmann,Tematisk,2022,"<p class=""col-md-6"">Mennesker med en psykisk l..."
4,Cultivating Women Tech Founders: A Communities...,2834347,Shuang Lu,Aarhus Universitet,Inge Lehmann,Tematisk,2022,"<p class=""col-md-6"">Flere kvindelige iværksætt..."
5,DarkDNA - unidentified eDNA sequences as ecolo...,2879406,Eva Egelyng Sigsgaard,Aarhus Universitet,Inge Lehmann,Tematisk,2022,"<p class=""col-md-6"">I den nuværende biodiversi..."
6,Digital Alternative News Use in Denmark (Alter...,2879859,Eva-Maria Laura Anna Mayerhöffer,Roskilde Universitet,Inge Lehmann,Tematisk,2022,"<p class=""col-md-6"">Projektet vil gennemføre d..."
7,Fibre in Foods: Wheat varieties for enhanced h...,2874486,Ourania Gouseti,Københavns Universitet,Inge Lehmann,Tematisk,2022,"<p class=""col-md-6"">Efterhånden som diætrelate..."
8,Finding the interaction point of single gamma ...,2879888,Rosana Martinez Turtos,Aarhus Universitet,Inge Lehmann,Tematisk,2022,"<p class=""col-md-6"">Scintillatorer er material..."
9,Functional ecosystem changes in the marine Arc...,2879782,Heike Hildegard Zimmermann,GEUS,Inge Lehmann,Tematisk,2022,"<p class=""col-md-6"">En af de største udfordrin..."
